In [1]:
from cobra.io import read_sbml_model
mod = read_sbml_model('C:/Users/prins/git/Human-GEM/model/Human-GEM.xml') 
import os
folder = "C:/Users/prins/git/Human1_RPE-PR/cs_mods"
files = [f for f in os.listdir(folder) if '.xml'in f if os.path.isfile(os.path.join(folder, f))]
models = {f:read_sbml_model(folder + '/' + f ) for f in files}
model_n = [f.split('_')[0] + f.split('_')[1] for f in files] 

In [2]:
model_n2 = ['Schwan (1)', 'Schwan (2)', 'Melanocytes', 'Endothelial','Muscle','Fibroblasts','RPE','B cells','T + NK cells',\
            'Monocytes + Macrophages','Mast','Retina','RPE + Choroid','PR rod','Amacrine','Bipolar',\
           'PR cone', 'Horizontal', 'Müller','Ganglion']
model_n2
model_n3 = list(range(1,21))

In [3]:
mod

Name,HumanGEM
Memory address,2627f269000
Number of metabolites,8369
Number of reactions,13070
Number of genes,3067
Number of groups,142
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [31]:
mod.reactions.get_by_id('MAR13082').build_reaction_string(use_metabolite_names = True)

'45.0 ATP + 0.0267 DNA + 45.0 H2O + 0.1124 RNA + 0.4062 glycogen + 0.0012 cofactor_pool_biomass + 5.3375 protein_pool_biomass + 0.2212 lipid_pool_biomass + 0.4835 metabolite_pool_biomass --> 45.0 ADP + 45.0 H+ + 45.0 Pi + biomass'

In [32]:
mod.reactions.get_by_id('MAR13082').reaction

'45.0 MAM01371c + 0.0267 MAM01721n + 45.0 MAM02040c + 0.1124 MAM02847c + 0.4062 MAM03161c + 0.0012 MAM10012c + 5.3375 MAM10013c + 0.2212 MAM10014c + 0.4835 MAM10015c --> 45.0 MAM01285c + 45.0 MAM02039c + 45.0 MAM02751c + MAM03970c'

In [5]:
mod = read_sbml_model('C:/Users/prins/git/Human-GEM/model/Human-GEM.xml') 
mod

Name,HumanGEM
Memory address,262ac7e3370
Number of metabolites,8369
Number of reactions,13070
Number of genes,3067
Number of groups,142
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [6]:
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()

In [8]:
def remove_compartment(model, compartment):
# remove compartment from model
    rxns_n = [r.id for r in model.reactions if compartment in r.compartments]
    mets_n = [m for m in model.metabolites if compartment in m.compartment]
    model.remove_reactions(rxns_n)
    model.remove_metabolites(mets_n)
    return model

# remove nucleus from PR model
mod_PR = remove_compartment(mod_PR, 'n')

In [9]:
def add_id_suffix(model, suffix):
    for r in model.reactions:
        r.id = r.id + suffix
    for m in model.metabolites:
        m.id = m.id + suffix
        m.compartment = m.compartment + suffix
    return model

# add '_PR' to PR model IDs and '_RPE' to RPE model IDs
mod_PR = add_id_suffix(mod_PR, '_PR')
mod_RPE = add_id_suffix(mod_RPE, '_RPE')

In [10]:
# add RPE-PR interface reactions to RPE model (rxn_RPE-PR = RPE <--> RPE-PR interface)
for rxns in  [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:     
    rxn = rxns.copy()
    rxn.id = rxn.id.replace('_RPE','_RPE-PR')
    for p in rxn.products:
        if 'e_RPE' in p.id:
            p.id = p.id.replace('e_RPE','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_RPE' in r.id:
            r.id = r.id.replace('e_RPE','e_RPE-PR')
            r.compartment = 'e_RPE-PR'
    mod_RPE.add_reactions([rxn])

In [11]:
# add RPE-PR interface reactions to PR model (rxn_PR-RPE = PR <--> RPE-PR interface)
for rxn in [r for r in mod_PR.reactions if 'e_PR' in r.reaction]:
    rxn.id = rxn.id.replace('_PR','_PR-RPE')
    for p in rxn.products:
        if 'e_PR' in p.id:
            p.id = p.id.replace('e_PR','e_RPE-PR')
            p.compartment = 'e_RPE-PR'
    for r in rxn.reactants:
        if 'e_PR' in r.id:
            r.id = r.id.replace('e_PR','e_RPE-PR')
            r.compartment = 'e_RPE-PR'

In [12]:
# fuse RPE and PR model
from cobra import Model
mod_RPE_PR = Model('RPE-PR model')
mod_RPE_PR.add_reactions(mod_RPE.reactions)
mod_RPE_PR.add_reactions(mod_PR.reactions)

In [13]:
# list reaction IDs to be deleted (because their reactions are duplicated)
import pandas as pd
df = pd.DataFrame([[r.id, r.reaction] for r in mod_RPE_PR.reactions \
                   if r.compartments == {'e_RPE-PR'}], columns=['id','reaction'])
l  = list(df['id'][df['reaction'].duplicated()])

# delete duplicated reactions and change reaction IDs (suffix: _eRPE-PR) 
# of the reactions that only involve the e_RPE-PR compartment (interface)
for r in [r for r in mod_RPE_PR.reactions if r.compartments=={'e_RPE-PR'}]: 
    if r.id in l:
        mod_RPE_PR.remove_reactions(r.id)
    elif '_RPE-PR' in r.id:
        r.id = r.id.replace('_RPE-PR','_eRPE-PR')
    elif '_PR-RPE' in r.id:
        r.id = r.id.replace('_PR-RPE','_eRPE-PR')

C:\Users\prins\AppData\Roaming\Python\Python310\site-packages\cobra\core\model.py:778: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [15]:
def get_vmh_id(reaction):
    if 'vmhreaction' in reaction.annotation.keys(): 
        return reaction.annotation['vmhreaction']
get_vmh_id(mod_RPE_PR.reactions[0])

'ALCD2if'

In [16]:
df= pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions],\
             columns=['id','vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])
df.to_clipboard(excel=True, sep=None)
df

,id,vmh id,name,subsystem,reaction (id),reaction (name)
0,MAR03905,ALCD2if,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01796c + MAM02552c --> MAM01249c + MAM02039...,ethanol + NAD+ --> acetaldehyde + H+ + NADH
1,MAR03907,ALCD2yf,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,MAM01796c + MAM02554c --> MAM01249c + MAM02039...,ethanol + NADP+ --> acetaldehyde + H+ + NADPH
2,MAR04097,ACS,Acetate:CoA ligase (AMP-forming),Glycolysis / Gluconeogenesis,MAM01252c + MAM01371c + MAM01597c --> MAM01261...,acetate + ATP + CoA --> acetyl-CoA + AMP + PPi
3,MAR04099,ACSm,Acetate:CoA ligase (AMP-forming),Glycolysis / Gluconeogenesis,MAM01252m + MAM01371m + MAM01597m --> MAM01261...,acetate + ATP + CoA --> acetyl-CoA + AMP + PPi
4,MAR04108,r0068,acetyl adenylate:CoA acetyltransferase,Glycolysis / Gluconeogenesis,MAM01257c + MAM01597c --> MAM01261c + MAM01334...,acetyl adenylate + CoA --> acetyl-CoA + AMP + H+
...,...,...,...,...,...,...
13065,MAR10132,None,,Exchange/demand reactions,MAM10041e <=>,tauro-hyodeoxycholic acid <=>
13066,MAR13083,None,,Metabolism of other amino acids,MAM02039c + MAM02157c + MAM02553c + MAM02630c ...,H+ + hypotaurine + NADH + O2 --> H2O + NAD+ + ...
13067,MAR13084,None,,Transport reactions,MAM01639c <=> MAM01639n,dAMP <=> dAMP
13068,MAR13085,None,,Transport reactions,MAM01686c <=> MAM01686n,dGMP <=> dGMP


In [17]:
import pandas as pd
df = pd.DataFrame([[r.id, get_vmh_id(r), r.name, r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
                   for r in mod_RPE_PR.reactions if r.compartments == {'e_RPE-PR'}], \
                   columns=['id','vmh id', 'name', 'reaction (id)', 'reaction (name)'])
df[~df['reaction (id)'].duplicated(keep=False)].to_clipboard(excel=True, sep=None)

In [18]:
df = pd.DataFrame([[r.id, get_vmh_id(r), r.name, r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
                   for r in mod_RPE_PR.reactions], \
                   columns=['id','vmh id', 'name', 'reaction (id)', 'reaction (name)'])
df_dup = pd.DataFrame([r.reaction for r in mod_RPE_PR.reactions]).duplicated(keep='first')

In [19]:
mod.reactions.get_by_id('MAR01862')

Reaction identifier,MAR01862
Name,
Memory address,0x262c17655d0
Stoichiometry,MAM01988e + MAM02519e --> MAM01988c + MAM02519c glycocholate + Na+ --> glycocholate + Na+
GPR,ENSG00000100652 or ENSG00000125255
Lower bound,0.0
Upper bound,1000.0


In [20]:
import pandas as pd
subsystems = list(set([r.subsystem for r in mod.reactions]))
df = pd.DataFrame(index=subsystems)
for i in range(len(files)):
    count = [[rxn.subsystem for rxn in models[files[i]].reactions].count(s) for s in subsystems]
    df[model_n2[i]] = count 
df

,Schwan (1),Schwan (2),Melanocytes,Endothelial,Muscle,Fibroblasts,RPE,B cells,T + NK cells,Monocytes + Macrophages,Mast,Retina,RPE + Choroid,PR rod,Amacrine,Bipolar,PR cone,Horizontal,Müller,Ganglion
Retinol metabolism,50,55,47,55,55,54,58,48,50,49,51,49,57,50,49,52,53,50,52,49
Beta oxidation of phytanic acid (peroxisomal),6,9,1,1,6,6,6,6,6,6,4,6,19,5,5,6,5,9,6,5
Acylglycerides metabolism,20,18,18,17,18,17,15,16,17,18,16,15,18,16,16,10,17,12,16,16
Riboflavin metabolism,4,1,4,3,4,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4
Ether lipid metabolism,4,8,8,8,8,8,8,8,8,8,8,8,8,7,8,7,7,7,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Alanine, aspartate and glutamate metabolism",17,19,18,18,17,14,16,10,11,11,10,14,17,14,16,16,15,17,13,15
Miscellaneous,35,24,26,26,24,23,37,19,28,21,25,26,33,24,18,24,23,23,24,25
Alkaloids biosynthesis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Omega-3 fatty acid metabolism,69,69,69,68,69,69,69,69,69,68,71,69,73,69,69,69,69,69,69,69


In [21]:
atp_rxns = [list(m.reactions) for m in mod.metabolites if 'ATP' == m.name] 
atp_rxns = list(set([i for atp_rxns_i in atp_rxns for i in atp_rxns_i])) # unique 
df_atp_rxns = pd.DataFrame([[r.id,\
              get_vmh_id(r), \
              r.name, \
              r.build_reaction_string(use_metabolite_names = True)]\
              for r in atp_rxns])
df_atp_rxns

,0,1,2,3
0,MAR02385,r1526,ATP-Binding Cassette (ABC) Tcdb:3.A.1.211.1,ATP + H2O + Gm4-Pool --> ADP + H+ + Pi + Gm4-Pool
1,MAR04727,PTPAT,ATP:pantetheine-4'-phosphate adenylyltransferase,ATP + H+ + phosphopantetheine <=> dephospho-Co...
2,MAR12260,HDCA6Gte,Glucuronidated bile acid excretion (HDCA-6G),"ATP + H2O + Hyodeoxycholic acid-6glucuronide, ..."
3,MAR10382,LEUKTRB4WOHtr,"Transport of W-Hydroxyl Leukotriene B4, Intrac...",20-OH-LTB4 + ATP + H2O --> 20-OH-LTB4 + ADP + ...
4,MAR00237,FACOAL170,,AMP + heptadecanoyl-CoA + PPi <=> ATP + CoA + ...
...,...,...,...,...
1178,MAR08478,DCK1n,ATP:deoxycitidine 5'-phosphotransferase,ADP + dCMP + H+ <=> ATP + deoxycytidine
1179,MAR02949,r0311,Palmitate:CoA ligase (AMP-forming),AMP + palmitoyl-CoA + PPi <=> ATP + CoA + palm...
1180,MAR00674,HMR_0674,ATP:acylglycerol 3-phosphotransferase,1-acylglycerol-LD-PC pool + ATP --> ADP + H+ +...
1181,MAR12493,AM1CCSteb,efflux of AM1c-cyclosporine into bile,ATP + H2O + AM1c (cyclosporine) --> ADP + H+ +...


In [22]:
len(atp_rxns)

1183

In [23]:
import pandas as pd
subsystems = list(set([r.subsystem for r in mod.reactions]))
df_h1 = pd.DataFrame(index=subsystems)
count = [[rxn.subsystem for rxn in mod.reactions].count(s) for s in subsystems]
df_h1['Human1'] = count 
df['Human1']=df_h1
df

,Schwan (1),Schwan (2),Melanocytes,Endothelial,Muscle,Fibroblasts,RPE,B cells,T + NK cells,Monocytes + Macrophages,...,Retina,RPE + Choroid,PR rod,Amacrine,Bipolar,PR cone,Horizontal,Müller,Ganglion,Human1
Retinol metabolism,50,55,47,55,55,54,58,48,50,49,...,49,57,50,49,52,53,50,52,49,86
Beta oxidation of phytanic acid (peroxisomal),6,9,1,1,6,6,6,6,6,6,...,6,19,5,5,6,5,9,6,5,21
Acylglycerides metabolism,20,18,18,17,18,17,15,16,17,18,...,15,18,16,16,10,17,12,16,16,21
Riboflavin metabolism,4,1,4,3,4,3,3,4,4,4,...,4,4,4,4,4,4,4,4,4,6
Ether lipid metabolism,4,8,8,8,8,8,8,8,8,8,...,8,8,7,8,7,7,7,7,8,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Alanine, aspartate and glutamate metabolism",17,19,18,18,17,14,16,10,11,11,...,14,17,14,16,16,15,17,13,15,40
Miscellaneous,35,24,26,26,24,23,37,19,28,21,...,26,33,24,18,24,23,23,24,25,85
Alkaloids biosynthesis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
Omega-3 fatty acid metabolism,69,69,69,68,69,69,69,69,69,68,...,69,73,69,69,69,69,69,69,69,75


In [24]:
df['Human1']=df_h1

df2=df[(df == 0).any(axis=1)] # select rows and columns containing zeros
idx_row = df2[df2==0].count(axis=1).sort_values(ascending=False).index.to_list()
idx_col = df2[df2==0].count(axis=0).sort_values(ascending=False).index.to_list()
df2 = df2.reindex(idx_col, axis=1).reindex(idx_row,axis=0)

def style_negative(v, props=''):
    return props if v == 0 else None
df2 = df2[(df == 0).any(axis=1)].style.applymap(style_negative, props='background-color:gray;').\
set_table_styles([dict(selector="th", props=[('width', '10px')]),
              dict(selector="th.col_heading",
                   props=[("writing-mode", "vertical-rl"),
                          ('transform', 'rotateZ(180deg)'), 
                          ('height', '170px'),
                          ('vertical-align', 'top')]),
              dict(selector="th.row_heading",
                   props=[('width', '450px'),])])
df2

C:\Users\prins\AppData\Local\Temp\ipykernel_55844\1533517839.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[(df == 0).any(axis=1)].style.applymap(style_negative, props='background-color:gray;').\


,B cells,Muscle,Mast,Melanocytes,Monocytes + Macrophages,Schwan (2),T + NK cells,Endothelial,Fibroblasts,Amacrine,Bipolar,Schwan (1),RPE,RPE + Choroid,Horizontal,Ganglion,PR cone,Müller,PR rod,Retina,Human1
CoA catabolism,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Dietary fiber binding,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12
Triacylglycerol synthesis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Hippurate metabolism,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
Alkaloids biosynthesis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
Vitamin B2 metabolism,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2
Peptide metabolism,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,132,0,1,242
Fatty acid biosynthesis (odd-chain),0,0,0,0,0,0,29,0,0,0,0,0,29,29,0,0,0,0,0,29,29
Biotin metabolism,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,0,0,14
Vitamin C metabolism,0,0,0,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,2


In [25]:
df_norm = df.drop(['Human1'], axis=1).divide(df['Human1'].values, axis='rows')
idx = df_norm.mean(axis=1).sort_values(ascending=False).index.to_list()
df_norm['Human1 (number of reactions)']=df['Human1']
df_norm=df_norm.reindex(idx,axis=0)
df_norm.to_clipboard(excel=True, sep=None)
df_norm

,Schwan (1),Schwan (2),Melanocytes,Endothelial,Muscle,Fibroblasts,RPE,B cells,T + NK cells,Monocytes + Macrophages,...,Retina,RPE + Choroid,PR rod,Amacrine,Bipolar,PR cone,Horizontal,Müller,Ganglion,Human1 (number of reactions)
Fatty acid activation (endoplasmic reticular),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,55
Fatty acid elongation (odd-chain),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,12
Fatty acid biosynthesis (unsaturated),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,32
Keratan sulfate biosynthesis,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,59
Beta oxidation of di-unsaturated fatty acids (n-6) (mitochondrial),1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dietary fiber binding,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12
CoA catabolism,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
Alkaloids biosynthesis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
Hippurate metabolism,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [30]:
[r.id for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis']
pd.DataFrame([[r.id, get_vmh_id(r), r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
              for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis'],\
             columns=['id','vhr id','name','reaction','reaction (names)']).to_clipboard(excel=True, sep=None)

In [27]:
rxns_mod = [r.id for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis']
l_bin_gly_rxns = [[1 if r in rxns_i else 0 for r in rxns_mod]\
                  for rxns_i in [[r.id for r in models[f].reactions\
                  if r.subsystem == 'Glycolysis / Gluconeogenesis'] for f in files]]
df_gly_rxns = pd.DataFrame([[r.id,r.name] for r in mod.reactions if r.subsystem == 'Glycolysis / Gluconeogenesis'], columns=['id','name'])
for i in range(len(model_n2)):
    df_gly_rxns[model_n2[i]]=l_bin_gly_rxns[i]

idx_row = df_gly_rxns[df_gly_rxns==0].count(axis=1).sort_values(ascending=True).index.to_list()
idx_col = df_gly_rxns[df_gly_rxns==0].count(axis=0).sort_values(ascending=True).index.to_list()
df_gly_rxns = df_gly_rxns.reindex(idx_col, axis=1).reindex(idx_row,axis=0)
df_gly_rxns.to_clipboard(excel=True, sep=None)
df_gly_rxns

,id,name,Schwan (1),RPE,Müller,Monocytes + Macrophages,Retina,Schwan (2),Bipolar,PR rod,...,Muscle,Amacrine,Ganglion,Melanocytes,Endothelial,PR cone,Fibroblasts,T + NK cells,B cells,Mast
18,MAR04368,ATP:3-phospho-D-glycerate 1-phosphotransferase,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
26,MAR04391,D-glyceraldehyde-3-phosphate aldose-ketose-iso...,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
23,MAR04375,"D-fructose-1,6-bisphosphate D-glyceraldehyde-3...",1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
22,MAR04373,D-glyceraldehyde-3-phosphate:NAD+ oxidoreducta...,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
21,MAR04372,"2,3-bisphospho-D-glycerate 2-phosphohydrolase",1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
20,MAR04371,"3-phospho-D-glycerate 1,2-phosphomutase",1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,MAR04281,(S)-Lactate:NAD+ oxidoreductase,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,MAR04388,(S)-Lactate:NAD+ oxidoreductase,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
17,MAR04365,"D-phosphoglycerate 2,3-phosphomutase",1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
13,MAR04355,"sedoheptulose 1,7-bisphosphate D-glyceraldehyd...",1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
